©2020-2021 ETH Zurich, Pagan Nicolò; D-ITET; Automatic Control Lab

This program is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with this program.  If not, see <https://www.gnu.org/licenses/>.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from pytictoc import TicToc
import tikzplotlib
import random
import csv

from functions import *
from simulation import Simulation, Simulation_parameters
from model import Model, Model_parameters

## Theoretical Outdegree Analysis

Plot of the theoretical probability density function, as a function of the network size N, for different values of N. 

In [ ]:
random.seed(0)
plt.rcParams["figure.figsize"]=[10,7]
tt=TicToc()
tt.tic()
N = [100, 1000, 10000, 100000, 1000000]
maxN = max(N)
# n_tol defines an approximation threshold: in the following the probability that a node has out-degree larger than n_tol is 0. In fact, the probability is typically
# really small, yet non zero. In principle, a node can have an out-degree up to N-1, where N is the size of the network, however this happens with probability 1/(N-1)!
# In practice, the probability for a node of having out-degree larger than n_tol can be assumed to be 0 for the following analysis.
n_tol = 50

folder = 'theoretical_outdegree_analysis/'
save_results = True
if not os.path.exists(folder):
    os.makedirs(folder)


outdegree_table, outdegree_table_cum = compute_outdegree_table_fast(maxN, n_tol)
expected_outdegree = compute_expected_outdegree(maxN)
sttdev_outdegree = compute_stddev_outdegree(maxN, outdegree_table, expected_outdegree, n_tol)

for i in range(len(N)):
    plt.plot(range(1,min(n_tol,N[i])), outdegree_table[N[i]-1, 1:min(n_tol,N[i])], label='N='+str(N[i]))
plt.xscale('log')
plt.yscale('log')
plt.xlim([1, n_tol])
plt.ylim([10**(-10),1])
plt.xlabel('N')
plt.ylabel('Outdegre pdf')
plt.title('Theoretical Outdegree distributions')
plt.legend(loc="upper right")
if (save_results):
     tikzplotlib.save(folder+'theoretical_outdegree_distributions.tikz',  table_row_sep='\\\\\n')
plt.show() 
tt.toc()

For the same values defined above, in what follow we provide th plot of the theoretical complementary cumulative distribution function, as a function of the network size N, for different values of N. 
The results are shown in Fig. 7a.

In [ ]:
for i in range(len(N)):
    outdegree_table_ccdf = 1-np.cumsum(outdegree_table[N[i]-1,0:min(n_tol,N[i])-1])
    plt.plot(range(1,min(n_tol,N[i])), outdegree_table_ccdf[0:min(n_tol,N[i])-1], label='N='+str(N[i]))
plt.xscale('log')
plt.yscale('log')
plt.xlim([1, 50])
plt.ylim([10**(-8),2])
plt.xlabel('N')
plt.ylabel('Outdegre ccdf')
plt.title('Theoretical Outdegree distributions')
plt.legend(loc="lower left")
if (save_results):
     tikzplotlib.save(folder+'theoretical_outdegree_distributions_ccdf.tikz', table_row_sep='\\\\\n')
plt.show()  

In the following, we compute and plot the expected out-degree as a function of the network size N. The results are shown in Fig. 7b.

In [ ]:
n = np.linspace(10,maxN,200, dtype=np.int64)
plt.errorbar(n, expected_outdegree[n-1], sttdev_outdegree[n-1], linestyle='None', marker='^', alpha=0.1)
plt.plot(n, np.log(n)/np.log(2), label='Log(2)');
plt.legend(loc="upper right")
plt.xlabel('N')
plt.ylabel('Expected Outdegree')
plt.title('Expected Outdegree')
if (save_results):
    with open(folder+'expected_outdegree.dat', 'w') as f:
        writer = csv.writer(f, delimiter='\t')
        writer.writerows(zip(n,expected_outdegree[n-1],sttdev_outdegree[n-1],sttdev_outdegree[n-1]))
    tikzplotlib.save(folder+'expected_outdegree.tikz',  table_row_sep='\\\\\n')
plt.show()